##Version Date: October 13 2023, Henrik Loecke

In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import math
import pandas as pd
import numpy as np
import datetime as dt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *
import subprocess

In [2]:
class Node():
    def __init__(self,node):    
        self.node = node
    def invert(self):        
        return node_df.loc[self.node,'Invert Level']
    def ground(self):        
        return node_df.loc[self.node,'Ground Level']
    def soh(self):        
        return node_df.loc[self.node,'Safe Operating Head']
    def asset_name(self):        
        return node_df.loc[self.node,'Asset Name']
    def cover(self):        
        return node_df.loc[self.node,'Cover Type']

class Pipe():
    def __init__(self,pipe):    
        self.pipe = pipe
    def height(self):        
        return pipe_df.loc[self.pipe,'Height'] 
    def uplevel(self):
        uplevel = pipe_df.loc[self.pipe,'UpLevel']
        if math.isnan(uplevel):
            node = pipe_df.loc[self.pipe,'FromNode']
            uplevel = node_df.loc[node,'Invert Level']
        return uplevel
    def dwlevel(self):
        dwlevel = pipe_df.loc[self.pipe,'DwLevel']
        if math.isnan(dwlevel):
            node = pipe_df.loc[self.pipe,'ToNode']
            dwlevel = node_df.loc[node,'Invert Level']
        return dwlevel
    

In [3]:
for m in master_list:
    
    [model_area,model,result_folder,output_folder,res_list,groupby_acronym_owner,pipe_ls_exclusions]
    
    model_area = m[0]
    model = m[1]
    result_folder = m[2]  
    output_folder = m[3]
    result_list = m[4]
    groupby_acronym_owner = m[5]
    pipe_ls_exclusions = m[6]
    name_shortenings = m[8]
    
    #Prepare button input
    buttons_ons = []
    result_groups = []
    result_groups.extend(result[2] for result in result_list if result[2] not in result_groups)
    button_ons_all = [True,True,True,True]
    for i, result_group in enumerate(result_groups):
        button_ons = [True,True,True,True]
        for result in result_list:
            if i == 0: 
                button_ons_all.append(True)
            if result[2] == result_group:
                button_ons.append(True)
            else:
                button_ons.append(False)
        if i == 0:
            buttons_ons.append(['All',button_ons_all])
        button_text = result_group
        for name_shortening in name_shortenings:
            button_text = button_text.replace(name_shortening[0],name_shortening[1])
        buttons_ons.append([button_text,button_ons])
        
    #Read model data
    parameter_script = r"Acronym_Crawler.py"
    bat_file_path = 'Read_Parameters.bat'
    bat_file = open(bat_file_path, "w")
    model_path = result_folder + '\\' + model
    bat_file.write('"' + parameter_script + '" "' + os.getcwd() + '" "' + model_path + '"')
    bat_file.close()
    result = subprocess.call([bat_file_path]) 
    if os.path.exists(model_path) == False:
        raise ValueError("The variable 'model_path' points to a path that does not exist: " + model_path)
    if result == 1: #Error
        raise ValueError("The sub process threw an error. Please Locate the bat file: " + bat_file_path + ", open it in notepad, \
        then add a new line and type in letters only: Pause. Double click the bat file to run it and it will show the error.")
       
    node_df = pd.read_csv(os.getcwd() + '\\ls_nodes.csv',dtype={'Node': str})
    node_df.set_index('Node',inplace=True)
    node_df['Asset Name'].fillna('',inplace=True)
    pipe_df = pd.read_csv(os.getcwd() + '\\ls_pipes.csv',dtype={'Pipe': str,'FromNode': str,'ToNode': str})
    pipe_df.set_index('Pipe',inplace=True)
    for pipe in pipe_ls_exclusions:
        pipe_df.drop(pipe,inplace=True)
  
    print('Loading results in ' + result_folder)
    for r in result_list:
        header = r[0]
        file = r[1]
        result_subfolder = ''
        #find subfolders for MIKE+
        if model[-7:] == '.sqlite':
            file_found = False
            for f1 in os.listdir(result_folder):
                if f1[-7:] == '.sqlite':
                    #browse subfolder              
                    try:
                        check_subfolder = os.path.basename(f1)[:-7] + '_m1d - Result Files'
                        for f2 in os.listdir(result_folder + '\\' + check_subfolder):
                            if os.path.basename(f2) == file:
                                result_subfolder = '\\' + check_subfolder
                                file_found = True
                    except:
                        pass
            if not file_found:
                message = 'The following result file was not found:\n' + file
                raise ValueError(message) 
                     
        res1d = Res1D(result_folder + result_subfolder + '\\' + file)
        print('Loading ' + file)
        r.append(res1d)
        
        
    missing_pipes = []

    node_delta_xs = [['US',-1],['DS',0]]

    acronyms = list(pipe_df.Acronym.unique())
    acronyms.sort()
    paths = []

    loop_counter = 0
    
    print ("Building path")

    for j, acronym in enumerate(acronyms):
        print ("Building path for " + acronym + " number " + str(j+1) + " of " + str(len(acronyms)) + " at " + str(dt.datetime.now()))
        
        path_no = 0

        pipe_index_df = pipe_df[pipe_df.Acronym==acronym].copy()

        while len(pipe_index_df)>0:


            start_pipes = []
            for index, row in pipe_index_df.iterrows():
                if row['FromNode'] not in list(pipe_index_df.ToNode):

                    start_pipes.append(index)

            start_pipes

            for start_pipe in start_pipes[:1]:
                exit_path = False
                chainage = 0

                node = pipe_index_df.loc[start_pipe,'FromNode']
                pipe = start_pipe

                path = []

                for node_delta_x in node_delta_xs:
                    point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            np.nan,np.nan]
                    for r in result_list:
                        values = r[3].query.GetNodeValues(node, "WaterLevel")
                        if values != None:
                            point.append(max(values))
                        else:
                            point.append(np.nan)                    
                    path.append(point)

                i = 0
                while exit_path == False and i < 1000:

                    point = [path_no,i*3+1,acronym,'Pipe US',pipe,chainage,Pipe(pipe).uplevel(),
                                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]

                    for r in result_list:                    
                        try:
                            values = r[3].query.GetReachStartValues(pipe, "WaterLevel")
                        except:
                            values = None
                        if values != None:
                            point.append(max(values))
                        else:
                            point.append(np.nan) 

                    path.append(point)

                    chainage += pipe_index_df.loc[pipe,'Length']

                    node = pipe_index_df.loc[pipe,'ToNode']


                    point = [path_no,i*3+1,acronym,'Pipe DS',pipe,chainage-1,Pipe(pipe).dwlevel(),
                                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()]

                    for r in result_list:
                        try:
                            values = r[3].query.GetReachEndValues(pipe, "WaterLevel")
                        except:
                            values = None
                        if values != None:
                            point.append(max(values))
                        else:
                            point.append(np.nan) 

                    path.append(point)


                    for node_delta_x in node_delta_xs:
                        point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                                     Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                    np.nan,np.nan]

                        for r in result_list:
                            values = r[3].query.GetNodeValues(node, "WaterLevel")
                            if values != None:
                                point.append(max(values))
                            else:
                                point.append(np.nan)                 
                        path.append(point)

                    pipe_index_df.drop(pipe,inplace=True)

                    try:
                        pipe = pipe_index_df[pipe_index_df.FromNode==node].index[0]
                    except:
                        exit_path = True   
                    i += 1

                    loop_counter += 1
                    if loop_counter > 20000:
                        raise


            paths += path

            path_no += 1

    # paths
    columns = ['Path Number','Sqn','Acronym','Type','MUID','Chainage','Invert Level','Ground Level','Safe Operating Head',
                                       'Asset Name','Cover Type','Height','Pipe Obvert']

    for r in result_list:
        header = r[0]
        columns.append(header)


    path_df=pd.DataFrame(paths,columns=columns)
    path_df.replace(999,np.nan,inplace=True)
    
    # output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plots'

    colors = ['black','black','green','red','brown','orange','grey','brown','midnightblue','tan','firebrick','darkturquoise','darkorchid','dimgray','darkgoldenrod','indianred','deeppink','AliceBlue','AntiqueWhite','Aqua','Aquamarine','Azure','Beige','Bisque','Black','BlanchedAlmond','Blue','BlueViolet','Brown','BurlyWood','CadetBlue','Chartreuse','Chocolate','Coral','CornflowerBlue','Cornsilk','Crimson','Cyan','DarkBlue','DarkCyan','DarkGoldenrod','DarkGray','DarkGreen','DarkGrey','DarkKhaki','DarkMagenta','DarkOliveGreen','DarkOrange','DarkOrchid','DarkRed','DarkSalmon','DarkSeaGreen','DarkSlateBlue','DarkSlateGray','DarkSlateGrey','DarkTurquoise','DarkViolet','DeepPink','DeepSkyBlue','DimGray','DodgerBlue','FireBrick','FloralWhite','ForestGreen','Fuchsia','Gainsboro','GhostWhite','Gold','Goldenrod','Gray','Green','GreenYellow','Grey','Honeydew','HotPink','IndianRed','Indigo','Ivory','Khaki','Lavender','LavenderBlush','LawnGreen','LemonChiffon','LightBlue','LightCoral','LightCyan','LightGoldenrodYellow','LightGray','LightGreen','LightGrey','LightPink','LightSalmon','LightSeaGreen','LightSkyBlue','LightSlateGray','LightSlateGrey','LightSteelBlue','LightYellow','Lime','LimeGreen','Linen','Magenta','Maroon','MediumAquamarine','MediumBlue','MediumOrchid','MediumPurple','MediumSeaGreen','MediumSlateBlue','MediumSpringGreen','MediumTurquoise','MediumVioletRed','MidnightBlue','MintCream','MistyRose','Moccasin','NavajoWhite','Navy','OldLace','Olive','OliveDrab','Orange','OrangeRed','Orchid','PaleGoldenrod','PaleGreen','PaleTurquoise','PaleVioletRed','PapayaWhip','PeachPuff','Peru','Pink','Plum','PowderBlue','Purple','Rebeccapurple','Red','RosyBrown','RoyalBlue','SaddleBrown','Salmon','SandyBrown','SeaGreen','Seashell','Sienna','Silver','SkyBlue','SlateBlue','SlateGray','SlateGrey','Snow','SpringGreen','SteelBlue','Tan','Teal','Thistle','Tomato','Turquoise','Violet','Wheat','White','WhiteSmoke','Yellow','YellowGreen']

    print ("Writing htmls under " + output_folder)
    for j, acronym in enumerate(acronyms):

        print ("Writing html for " + acronym + " number " + str(j+1) + " of " + str(len(acronyms)) + " at " + str(dt.datetime.now()))

        path_acronym_df = path_df[path_df.Acronym==acronym]

        output_subfolders = [output_folder + "\\All_Longsections"]
        if groupby_acronym_owner == True:
            output_subfolders.append(output_folder + '\\GV_Acronym_' + acronym)

        for i, output_subfolder in enumerate(output_subfolders):

            relative_output_subfolder = '..\\' + os.path.basename(output_subfolder)

            if not os.path.isdir(output_subfolder): os.makedirs(output_subfolder) 

            with open(output_subfolder + "\\Long_Section_" + acronym + ".html", 'w') as f:
                f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style.css">\n')

                f.write('<!DOCTYPE html>\n')
                f.write('<html>\n')
                f.write('<head>\n')
                f.write('<meta charset="utf-8">\n')
                f.write('</head>\n')
                f.write('<body>\n')
                f.write('<div class="sidenav">\n')


                f.write("<h2>Links to Others (Alphabetic Order)</h2>\n")
                if j == 0:
                    previous_acronym = acronyms[len(acronyms)-1]
                else:
                    previous_acronym = acronyms[j-1]
                if j == len(acronyms)-1:
                    next_acronym = acronyms[0]
                else:
                    next_acronym = next_acronym = acronyms[j+1]

                if i == 0:
                    path = relative_output_subfolder +  "\\Long_Section_" + previous_acronym + ".html"
                else:                
                    path = '..\\GV_Acronym_' + previous_acronym + "\\Long_Section_" + previous_acronym + ".html"

                f.write('<a href="' + path + '">Previous (' + previous_acronym + ')</a>\n')
                f.write("<br>")

                if i == 0:
                    path = relative_output_subfolder +  "\\Long_Section_" + next_acronym + ".html"
                else:                
                    path = '..\\GV_Acronym_' + next_acronym + "\\Long_Section_" + next_acronym + ".html"

                f.write('<a href="' + path + '">Next (' + next_acronym + ')</a>\n')
                f.write("<br>")            

                f.write("<h2>Map</h2>\n")

                map_string = '<img src="..\\Maps_And_CSS\\Acronym_' + acronym + '.jpg" alt="' + acronym + '">\n'
                f.write(map_string + "\n")
                f.write("<p></p>\n")

                f.write('</div>')
                f.write('<div class="main">\n')
                f.write("<h1>Long Sections for " + acronym + "</h1>\n")

                for path_no in path_acronym_df['Path Number'].unique():
                    path_acronym_sub_df = path_acronym_df[path_acronym_df['Path Number']==path_no]
                    path_acronym_node_df = path_acronym_sub_df[path_acronym_sub_df.Type=='Node-DS']

                    fig = go.Figure()

                    columns=['Invert Level','Pipe Obvert','Ground Level','Safe Operating Head']
                    for r in result_list:
                        header = r[0]
                        columns.append(header)

                    for i, column in enumerate(columns):
                        if i < 4:
                            line_width = 4
                        else:
                            line_width = 2
                        fig.add_trace(go.Scatter(x=path_acronym_sub_df.Chainage, 
                                                 y = path_acronym_sub_df[column],line_color=colors[i],name=column, line_width = line_width))

                    lbl_x = []
                    lbl_y = []
                    lbl_text = []

                    label_y = max(path_acronym_node_df['Ground Level']) + 2

                    for index, row in path_acronym_node_df.iterrows():

                        fig.add_shape(type="rect",x0=row['Chainage']-1, y0=row['Invert Level'], x1=row['Chainage'],
                                      y1=row['Ground Level'],line=dict(color="RoyalBlue",width=1),)

                        fig.add_shape(type="line",
                            x0=row['Chainage']-0.5, y0=row['Ground Level'], x1=row['Chainage']-0.5, y1=label_y-0.8,
                            line=dict(
                                color="Grey",
                                width=1,
                                dash="dot",
                            )
                        )

                        lbl_x.append(row['Chainage'] - 0.5)

                        lbl_y.append(label_y)
                        if row['Cover Type']==2:
                            cover_type = 'Sealed'
                        else:
                            cover_type = 'Unsealed'
                        lbl_text.append(str(row['Asset Name']) + '<br>' +  str(row['MUID']) + '<br>' + cover_type)

                    fig.add_trace(go.Scatter(
                        x=lbl_x,
                        y=lbl_y,
                        text=lbl_text,
                        mode="text",
                        name="Labels"
                    ))

                    fig.update_layout(
                        autosize=False,
                        width = 1362,
                        height=700,
                        margin=dict(
                            l=50,
                            r=50,
                            b=25,
                            t=35,
                            pad=4
                            ),
                        yaxis_title='Geodetic Elevation(m)',
                        xaxis_title='Chainage (m)',
                        

                        updatemenus=[
                            {
                                'buttons': [
                                    {
                                        'args': [{'visible': button_on[1]}, {'title': button_on[0]}],
                                        'label': button_on[0],
                                        'method': 'update'
                                    }
                                    for button_on in buttons_ons
                                ],
                                'direction': 'left',
                                'pad': {'r': 10, 't': 87},
                                'showactive': True,
                                'type': 'buttons',
                                'x': 0.1,
                                'xanchor': 'left',
                                'y': 0.06,
                                'yanchor': 'top'
                            }
                            ]                                           
                        )

                    f.write("<h2>Long Section " + str(path_no+1) + "</h2>\n")
                    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

                f.write('</div>')
                f.write('</body>')
                f.write('</html>')
                f.close()

Loading results in J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Model
Loading FSA_GA_EX-2y-24h-AES_2021p_Base-DSS1Default_Network_HD.res1d
Loading FSA_GA_EX-5y-24h-AES_2021p_Base-DSS2Default_Network_HD.res1d
Loading FSA_GA_EX-10y-24h-AES_2021p_Base-DSS3Default_Network_HD.res1d
Loading FSA_GA_EX-25y-24h-AES_2021p_Base-DSS16Default_Network_HD.res1d
Loading FSA_DWF_2021-07-22_4d_2021pop_BaseDefault_Network_HD.res1d
Loading FSA_WWF_2020-01-29_6d_2021pop_BaseDefault_Network_HD.res1d
Loading FSA_WWF_2021-11-12_5d_2021pop_BaseDefault_Network_HD.res1d
Building path
Building path for ALT number 1 of 205 at 2024-10-08 07:05:45.497025
Building path for ALX number 2 of 205 at 2024-10-08 07:05:45.998419
Building path for BBA number 3 of 205 at 2024-10-08 07:05:46.930890
Building path for BBB number 4 of 205 at 2024-10-08 07:05:47.702758
Building path for BBC number 5 of 205 at 2024-10-08 07:05:48.715308
Building path for BBL number 6 of 205 at 2024-10-08 07

Building path for NWA number 111 of 205 at 2024-10-08 07:07:04.491934
Building path for NWC number 112 of 205 at 2024-10-08 07:07:05.205810
Building path for NWD number 113 of 205 at 2024-10-08 07:07:06.659314
Building path for NWF number 114 of 205 at 2024-10-08 07:07:07.195280
Building path for NWG number 115 of 205 at 2024-10-08 07:07:08.241349
Building path for NWP number 116 of 205 at 2024-10-08 07:07:08.682448
Building path for NWR number 117 of 205 at 2024-10-08 07:07:09.013272
Building path for NWS number 118 of 205 at 2024-10-08 07:07:10.108073
Building path for NWT number 119 of 205 at 2024-10-08 07:07:10.962066
Building path for NWW number 120 of 205 at 2024-10-08 07:07:11.332879
Building path for NWX number 121 of 205 at 2024-10-08 07:07:12.355046
Building path for NXA number 122 of 205 at 2024-10-08 07:07:12.906168
Building path for NXB number 123 of 205 at 2024-10-08 07:07:12.954284
Building path for NXC number 124 of 205 at 2024-10-08 07:07:12.996372
Building path for NX

Writing html for CI2 number 23 of 205 at 2024-10-08 07:08:16.249744
Writing html for CIN number 24 of 205 at 2024-10-08 07:08:17.153017
Writing html for CMP number 25 of 205 at 2024-10-08 07:08:17.412743
Writing html for CQC number 26 of 205 at 2024-10-08 07:08:17.451718
Writing html for CQD number 27 of 205 at 2024-10-08 07:08:18.026126
Writing html for CQL number 28 of 205 at 2024-10-08 07:08:20.799753
Writing html for CQM number 29 of 205 at 2024-10-08 07:08:21.398290
Writing html for CTO number 30 of 205 at 2024-10-08 07:08:21.789520
Writing html for CTS number 31 of 205 at 2024-10-08 07:08:21.852025
Writing html for CV6 number 32 of 205 at 2024-10-08 07:08:22.248460
Writing html for CV7 number 33 of 205 at 2024-10-08 07:08:22.446048
Writing html for CVS number 34 of 205 at 2024-10-08 07:08:22.702040
Writing html for CZS number 35 of 205 at 2024-10-08 07:08:23.223805
Writing html for EF3 number 36 of 205 at 2024-10-08 07:08:23.302475
Writing html for FRP number 37 of 205 at 2024-10

Writing html for OSR number 144 of 205 at 2024-10-08 07:09:14.927602
Writing html for P2F number 145 of 205 at 2024-10-08 07:09:15.824472
Writing html for PFM number 146 of 205 at 2024-10-08 07:09:21.867371
Writing html for PIF number 147 of 205 at 2024-10-08 07:09:23.973850
Writing html for PIM number 148 of 205 at 2024-10-08 07:09:24.353734
Writing html for PMA number 149 of 205 at 2024-10-08 07:09:24.957902
Writing html for POC number 150 of 205 at 2024-10-08 07:09:25.086582
Writing html for PQE number 151 of 205 at 2024-10-08 07:09:25.676605
Writing html for PQO number 152 of 205 at 2024-10-08 07:09:25.816934
Writing html for PSE number 153 of 205 at 2024-10-08 07:09:25.883073
Writing html for PSM number 154 of 205 at 2024-10-08 07:09:26.141979
Writing html for PSO number 155 of 205 at 2024-10-08 07:09:26.993047
Writing html for PSS number 156 of 205 at 2024-10-08 07:09:27.224608
Writing html for PSW number 157 of 205 at 2024-10-08 07:09:27.265158
Writing html for QBF number 158 of